# Datasets

A dataset is a list of (input, target) pairs that can be further split into training and testing lists.

Let's make an example network to use as demonstration. This network will compute whether the number of 1's in a set of 5 bits is odd.

In [1]:
from conx import Network, Layer

net = Network("Odd Network")
net.add(Layer("input", 5))
net.add(Layer("hidden", 10, activation="relu"))
net.add(Layer("output", 1, activation="sigmoid"))
net.connect()
net.compile(error="mse", optimizer="adam")
net.summary()

Using Theano backend.


Network Summary
---------------
Network name: Odd Network
    Layer name: 'input' (input)
        VShape: None
        Dropout: 0
        Connected to: ['hidden']
        Activation function: None
        Dropout percent: 0
    Layer name: 'hidden' (hidden)
        VShape: None
        Dropout: 0
        Connected to: ['output']
        Activation function: relu
        Dropout percent: 0
    Layer name: 'output' (output)
        VShape: None
        Dropout: 0
        Activation function: sigmoid
        Dropout percent: 0


## As a list of (input, target) pairs

The most straightforward method of adding input, target vectors to train on is to use a list of (input, target) pairs. First we define a function that takes a number and returns the bitwise representation of it:

In [2]:
def num2bin(i, bits=5):
    """
    Take a number and turn it into a list of bits (most significant first).
    """
    return [int(s) for s in (("0" * bits) + bin(i)[2:])[-bits:]]

In [3]:
num2bin(23)

[1, 0, 1, 1, 1]

Now we make a list of (input, target) pairs:

In [4]:
patterns = []

for i in range(2 ** 5):
    inputs = num2bin(i)
    targets = [int(sum(inputs) % 2 == 1.0)]
    patterns.append((inputs, targets))

Pair set 5 looks like:

In [5]:
patterns[5]

([0, 0, 1, 0, 1], [0])

We set the network to use this dataset:

In [6]:
net.set_dataset(patterns)

In [7]:
net.dataset.summary()

Input Summary:
   count  : 32 (32 for training, 0 for testing)
   shape  : (5,)
   range  : (0.0, 1.0)
Target Summary:
   count  : 32 (32 for training, 0 for testing)
   shape  : (1,)
   range  : (0.0, 1.0)


## Dataset.add()

You can use the default `dataset` and add one pattern at a time. Consider the task of training a network to determine if the number of inputs is even (0) or odd (1). We could add inputs one at a time:

In [8]:
net.dataset.clear()

In [9]:
net.dataset.add([0, 0, 0, 0, 1], [1])
net.dataset.add([0, 0, 0, 1, 1], [0])
net.dataset.add([0, 0, 1, 0, 0], [1])

In [10]:
net.dataset.clear()

In [11]:
for i in range(2 ** 5):
    inputs = num2bin(i)
    targets = [int(sum(inputs) % 2 == 1.0)]
    net.dataset.add(inputs, targets)

In [12]:
net.dataset.summary()

Input Summary:
   count  : 32 (32 for training, 0 for testing)
   shape  : (5,)
   range  : (0.0, 1.0)
Target Summary:
   count  : 32 (32 for training, 0 for testing)
   shape  : (1,)
   range  : (0.0, 1.0)


In [13]:
net.dataset.inputs[13]

[0.0, 1.0, 1.0, 0.0, 1.0]

In [14]:
net.dataset.targets[13]

[1.0]

In [15]:
net.train(epochs=5000, accuracy=.75, tolerance=.2, report_rate=100)

Training...
Epoch #  100 | train error 0.24855 | train accuracy 0.56250 | validate% 0.00000
Epoch #  200 | train error 0.23902 | train accuracy 0.59375 | validate% 0.00000
Epoch #  300 | train error 0.23111 | train accuracy 0.62500 | validate% 0.00000
Epoch #  400 | train error 0.22265 | train accuracy 0.68750 | validate% 0.00000
Epoch #  500 | train error 0.21134 | train accuracy 0.68750 | validate% 0.00000
Epoch #  600 | train error 0.20184 | train accuracy 0.68750 | validate% 0.00000
Epoch #  700 | train error 0.18983 | train accuracy 0.75000 | validate% 0.00000
Epoch #  800 | train error 0.17956 | train accuracy 0.81250 | validate% 0.00000
Epoch #  900 | train error 0.17026 | train accuracy 0.81250 | validate% 0.00000
Epoch # 1000 | train error 0.16050 | train accuracy 0.87500 | validate% 0.06250
Epoch # 1100 | train error 0.15127 | train accuracy 0.90625 | validate% 0.09375
Epoch # 1200 | train error 0.14249 | train accuracy 0.93750 | validate% 0.09375
Epoch # 1300 | train error 0

In [16]:
net.test(tolerance=.2)

Testing on training dataset...
# | inputs | targets | outputs | result
---------------------------------------
0 | [0.0, 0.0, 0.0, 0.0, 0.0] | [0.0] | [0.1] | correct
1 | [0.0, 0.0, 0.0, 0.0, 1.0] | [1.0] | [0.9] | correct
2 | [0.0, 0.0, 0.0, 1.0, 0.0] | [1.0] | [0.8] | X
3 | [0.0, 0.0, 0.0, 1.0, 1.0] | [0.0] | [0.2] | correct
4 | [0.0, 0.0, 1.0, 0.0, 0.0] | [1.0] | [0.8] | correct
5 | [0.0, 0.0, 1.0, 0.0, 1.0] | [0.0] | [0.2] | correct
6 | [0.0, 0.0, 1.0, 1.0, 0.0] | [0.0] | [0.0] | correct
7 | [0.0, 0.0, 1.0, 1.0, 1.0] | [1.0] | [0.8] | correct
8 | [0.0, 1.0, 0.0, 0.0, 0.0] | [1.0] | [0.9] | correct
9 | [0.0, 1.0, 0.0, 0.0, 1.0] | [0.0] | [0.3] | X
10 | [0.0, 1.0, 0.0, 1.0, 0.0] | [0.0] | [0.5] | X
11 | [0.0, 1.0, 0.0, 1.0, 1.0] | [1.0] | [0.5] | X
12 | [0.0, 1.0, 1.0, 0.0, 0.0] | [0.0] | [0.1] | correct
13 | [0.0, 1.0, 1.0, 0.0, 1.0] | [1.0] | [0.9] | correct
14 | [0.0, 1.0, 1.0, 1.0, 0.0] | [1.0] | [0.8] | correct
15 | [0.0, 1.0, 1.0, 1.0, 1.0] | [0.0] | [0.2] | correct
16 | [1.0, 

## Dataset()

You can also create, and switch between, independent datasets.

In [17]:
from conx import Dataset

In [18]:
ds = Dataset()

In [19]:
for i in range(2 ** 5):
    inputs = num2bin(i)
    targets = [int(sum(inputs) % 2 == 1.0)]
    ds.add(inputs, targets)

As before, you can set the dataset, but using the Datatset object rather than list of (input, target) pairs:

In [20]:
net.set_dataset(ds)

In [21]:
net.test(tolerance=.2)

Testing on training dataset...
# | inputs | targets | outputs | result
---------------------------------------
0 | [0.0, 0.0, 0.0, 0.0, 0.0] | [0.0] | [0.1] | correct
1 | [0.0, 0.0, 0.0, 0.0, 1.0] | [1.0] | [0.9] | correct
2 | [0.0, 0.0, 0.0, 1.0, 0.0] | [1.0] | [0.8] | X
3 | [0.0, 0.0, 0.0, 1.0, 1.0] | [0.0] | [0.2] | correct
4 | [0.0, 0.0, 1.0, 0.0, 0.0] | [1.0] | [0.8] | correct
5 | [0.0, 0.0, 1.0, 0.0, 1.0] | [0.0] | [0.2] | correct
6 | [0.0, 0.0, 1.0, 1.0, 0.0] | [0.0] | [0.0] | correct
7 | [0.0, 0.0, 1.0, 1.0, 1.0] | [1.0] | [0.8] | correct
8 | [0.0, 1.0, 0.0, 0.0, 0.0] | [1.0] | [0.9] | correct
9 | [0.0, 1.0, 0.0, 0.0, 1.0] | [0.0] | [0.3] | X
10 | [0.0, 1.0, 0.0, 1.0, 0.0] | [0.0] | [0.5] | X
11 | [0.0, 1.0, 0.0, 1.0, 1.0] | [1.0] | [0.5] | X
12 | [0.0, 1.0, 1.0, 0.0, 0.0] | [0.0] | [0.1] | correct
13 | [0.0, 1.0, 1.0, 0.0, 1.0] | [1.0] | [0.9] | correct
14 | [0.0, 1.0, 1.0, 1.0, 0.0] | [1.0] | [0.8] | correct
15 | [0.0, 1.0, 1.0, 1.0, 1.0] | [0.0] | [0.2] | correct
16 | [1.0, 

## Dataset inputs and targets

Inputs and targets in the dataset are represented (both in accessing and in assignemnt) in the same format as given (as lists, or lists of lists). These formats are automattically converted into an internal format.

In [22]:
ds.inputs[17]

[1.0, 0.0, 0.0, 0.0, 1.0]

In [23]:
ds.inputs[17] = [0.9, 0.1, 0.1, 0.1, 0.9]

In [24]:
net.test(tolerance=.2)

Testing on training dataset...
# | inputs | targets | outputs | result
---------------------------------------
0 | [0.0, 0.0, 0.0, 0.0, 0.0] | [0.0] | [0.1] | correct
1 | [0.0, 0.0, 0.0, 0.0, 1.0] | [1.0] | [0.9] | correct
2 | [0.0, 0.0, 0.0, 1.0, 0.0] | [1.0] | [0.8] | X
3 | [0.0, 0.0, 0.0, 1.0, 1.0] | [0.0] | [0.2] | correct
4 | [0.0, 0.0, 1.0, 0.0, 0.0] | [1.0] | [0.8] | correct
5 | [0.0, 0.0, 1.0, 0.0, 1.0] | [0.0] | [0.2] | correct
6 | [0.0, 0.0, 1.0, 1.0, 0.0] | [0.0] | [0.0] | correct
7 | [0.0, 0.0, 1.0, 1.0, 1.0] | [1.0] | [0.8] | correct
8 | [0.0, 1.0, 0.0, 0.0, 0.0] | [1.0] | [0.9] | correct
9 | [0.0, 1.0, 0.0, 0.0, 1.0] | [0.0] | [0.3] | X
10 | [0.0, 1.0, 0.0, 1.0, 0.0] | [0.0] | [0.5] | X
11 | [0.0, 1.0, 0.0, 1.0, 1.0] | [1.0] | [0.5] | X
12 | [0.0, 1.0, 1.0, 0.0, 0.0] | [0.0] | [0.1] | correct
13 | [0.0, 1.0, 1.0, 0.0, 1.0] | [1.0] | [0.9] | correct
14 | [0.0, 1.0, 1.0, 1.0, 0.0] | [1.0] | [0.8] | correct
15 | [0.0, 1.0, 1.0, 1.0, 1.0] | [0.0] | [0.2] | correct
16 | [1.0, 

To see/access the internal format, use the underscore before inputs or targets. This is a numpy array. conx is designed so that you need not have to use numpy for most network operations.

In [25]:
ds._inputs[17]

array([ 0.89999998,  0.1       ,  0.1       ,  0.1       ,  0.89999998], dtype=float32)

## Built-in datasets

In [26]:
mnist = Dataset.get('mnist')

In [27]:
cifar10 = Dataset.get('cifar10')

In [28]:
cifar100 = Dataset.get('cifar100')

## Dataset operations

Dataset.split() will divide the dataset between training and testing sets. You can provide split an integer (to divide at a specific point), or a floating-point value, to divide by a percentage.

In [29]:
ds.split(20)

In [30]:
ds.split(.5)

In [31]:
ds.slice(10)

In [32]:
ds.shuffle()

### Additional operations

These functions are subject to change to an API which is more general:

In [ ]:
ds.set_targets_from_inputs()

In [ ]:
ds.set_inputs_from_targets()

In [ ]:
ds.set_targets_from_labels()

In [ ]:
ds.reshape_inputs(shape)

## Dataset direct manipulation

You can also set the internal format directly, given that it is in the correct format:

* use list of columns for multi-bank inputs or targets
* use np.array(vectors) for single-bank inputs or targets

In [34]:
import numpy as np

inputs = []
targets = []

for i in range(2 ** 5):
    v = num2bin(i)
    inputs.append(v)
    targets.append([int(sum(v) % 2 == 1.0)])

ds.load_direct(np.array(inputs), np.array(targets))

In [35]:
net.set_dataset(ds)

In [36]:
net.test(tolerance=.2)

Testing on training dataset...
# | inputs | targets | outputs | result
---------------------------------------
0 | [0.0, 0.0, 0.0, 0.0, 0.0] | [0.0] | [0.1] | correct
1 | [0.0, 0.0, 0.0, 0.0, 1.0] | [1.0] | [0.9] | correct
2 | [0.0, 0.0, 0.0, 1.0, 0.0] | [1.0] | [0.8] | X
3 | [0.0, 0.0, 0.0, 1.0, 1.0] | [0.0] | [0.2] | correct
4 | [0.0, 0.0, 1.0, 0.0, 0.0] | [1.0] | [0.8] | correct
5 | [0.0, 0.0, 1.0, 0.0, 1.0] | [0.0] | [0.2] | correct
6 | [0.0, 0.0, 1.0, 1.0, 0.0] | [0.0] | [0.0] | correct
7 | [0.0, 0.0, 1.0, 1.0, 1.0] | [1.0] | [0.8] | correct
8 | [0.0, 1.0, 0.0, 0.0, 0.0] | [1.0] | [0.9] | correct
9 | [0.0, 1.0, 0.0, 0.0, 1.0] | [0.0] | [0.3] | X
10 | [0.0, 1.0, 0.0, 1.0, 0.0] | [0.0] | [0.5] | X
11 | [0.0, 1.0, 0.0, 1.0, 1.0] | [1.0] | [0.5] | X
12 | [0.0, 1.0, 1.0, 0.0, 0.0] | [0.0] | [0.1] | correct
13 | [0.0, 1.0, 1.0, 0.0, 1.0] | [1.0] | [0.9] | correct
14 | [0.0, 1.0, 1.0, 1.0, 0.0] | [1.0] | [0.8] | correct
15 | [0.0, 1.0, 1.0, 1.0, 1.0] | [0.0] | [0.2] | correct
16 | [1.0, 